In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

## Function structure

### wrsamp(sig, fields)



- wrsamp will fill in default values for: nsig, siglen, basetime, fmt, if digital: (gain, baseline, units), . When a default value is filled in, print a warning message which states whether it was filled in because it was missing or because a requirement. 


Seems very difficult to define a 'isrequired' field in a class. Very convoluted. 



- Call wrheader(). At this point, the checksum, initvalue, res, nsig, siglen, fmt  have to already be calculated/present. 
  
  
- wrheader WILL check for validity of fields, and for the presence of (valid) requirements. We will not let it write invalid headers. It will NOT fill in default values, neither the calculated nor the non calculated ones. Summary: wrheader makes NO assumptions. 





recfields = OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True, None])),
                         ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False, None])),
                         ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True, None])),
                         ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True, None])),
                         ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False, None])),
                         ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False, None])),
                         ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True, None])),
                         ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False, None])),
                         ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False, None]))])

sigfields = OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True, None])),
                         ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True, None])),
                         ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                         ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                         ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                         ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                         ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                         ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                         ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                         ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                         ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))])
    

segfields = OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                         ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))])

comfields = OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])



In [30]:
from collections import OrderedDict
fields = OrderedDict([('height', 170), ('weight', 70), ('age', 20)])

for f in fields:
    print(f)

height
weight
age


In [24]:
import numpy as np

x=np.array([[1,3],[99,20],[8,8]])
y=x[:,0]
y.sort()
abs(np.diff(y))
max(y)

99

In [35]:
from collections import OrderedDict
fields = OrderedDict({'height': 170, 'weight': 70, 'age': 20})

for f in fields:
    print(f)

age
height
weight


In [7]:
fields = {'height': 170, 'weight': 70, 'age': 20}

for i in reversed(list(fields)):
    print(i)

weight
height
age


In [38]:
bitlevels=np.power(2, np.array(range(1, 33)))

nlevels=13000

bitlevels<nlevels
print(bitlevels)
np.argmax(bitlevels>13000)

[         2          4          8         16         32         64
        128        256        512       1024       2048       4096
       8192      16384      32768      65536     131072     262144
     524288    1048576    2097152    4194304    8388608   16777216
   33554432   67108864  134217728  268435456  536870912 1073741824
 2147483648 4294967296]


13

In [4]:
WFDBfields = ['height', 'weight', 'age']

final = dict.fromkeys(WFDBfields, [])
final

{'age': [], 'height': [], 'weight': []}

In [18]:
help(dict.fromkeys)

Help on built-in function fromkeys:

fromkeys(iterable, value=None, /) method of builtins.type instance
    Returns a new dict with keys from iterable and values equal to value.



In [5]:
for i in reversed(WFDBfields):
    print(i)

age
weight
height


In [57]:
a=[{'recname': 'wave1'},{'siglen': 500},{'fs': 125}]

In [1]:
# A field of the WFDB header file
from collections import OrderedDict

class WFDBfield():
    
    def __init__(self, speclist):
        # Name
        self.name = speclist[0] 
        
        # Data types the field can take
        self.allowedtypes = speclist[1]
        
        # The text delimiter that preceeds the field
        self.delimiter = speclist[2]
        
        # The required/dependent field which must also be present
        self.field_req = speclist[3]

        # Whether the field is mandatory specified by the WFDB guidelines
        self.is_req = speclist[4]
        
        # Whether the field is mandatory for writing (extra rules enforced by this library).
        # Being required for writing is not the same as the user having to specify. There are defaults. 
        self.write_req = speclist[5]

In [67]:
# 1. Record specification fields
recfields = OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True, None])),
                         ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False, None])),
                         ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True, None])),
                         ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True, None])),
                         ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False, None])),
                         ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False, None])),
                         ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True, None])),
                         ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False, None])),
                         ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False, None]))])
    
# 2. Signal specification fields
sigfields = OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True, None])),
                         ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True, None])),
                         ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                         ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                         ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                         ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                         ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                         ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                         ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                         ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                         ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))])
    
# 3. Segment specification fields
segfields = OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                         ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))])
    
# 4. Comment fields
comfields = OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])


In [69]:
WFDBfields=[recfields, sigfields, segfields, comfields]

[f for f in WFDBfields]

[OrderedDict([('recordname', <__main__.WFDBfield at 0x7fcc74397cf8>),
              ('nseg', <__main__.WFDBfield at 0x7fcc74397d30>),
              ('nsig', <__main__.WFDBfield at 0x7fcc74397d68>),
              ('fs', <__main__.WFDBfield at 0x7fcc74397da0>),
              ('counterfreq', <__main__.WFDBfield at 0x7fcc74397dd8>),
              ('basecounter', <__main__.WFDBfield at 0x7fcc74397e10>),
              ('siglen', <__main__.WFDBfield at 0x7fcc74397e48>),
              ('basetime', <__main__.WFDBfield at 0x7fcc74397e80>),
              ('basedate', <__main__.WFDBfield at 0x7fcc74397eb8>)]),
 OrderedDict([('filename', <__main__.WFDBfield at 0x7fcc74397cc0>),
              ('fmt', <__main__.WFDBfield at 0x7fcc74397c88>),
              ('sampsperframe', <__main__.WFDBfield at 0x7fcc74397f28>),
              ('skew', <__main__.WFDBfield at 0x7fcc74397f60>),
              ('byteoffset', <__main__.WFDBfield at 0x7fcc74397f98>),
              ('adcgain', <__main__.WFDBfield at 0x7fcc7

In [3]:
f2=WFDBfields[1]

In [7]:
f3=f1.copy()
f3.update(f2)

In [71]:
import sys
sys.exit("Missing input fields['nseg'] for writing multi-segment header.\nFor writing regular single segment headers, use the 'wrheader' function.")

SystemExit: Missing input fields['nseg'] for writing multi-segment header.
For writing regular single segment headers, use the 'wrheader' function.

To exit: use 'exit', 'quit', or Ctrl-D.


In [35]:
# 1. Record specification fields
recfields = OrderedDict([('recordname', WFDBfield(['recordname', [str], '', None, True, True, None])),
                         ('nseg', WFDBfield(['nseg', [int], '/', 'recordname', False, False, None])),
                         ('nsig', WFDBfield(['nsig', [int], ' ', 'recordname', True, True, None])),
                         ('fs', WFDBfield(['fs', [int, float], ' ', 'nsig', False, True, None])),
                         ('counterfreq', WFDBfield(['counterfreq', [int, float], '/', 'fs', False, False, None])),
                         ('basecounter', WFDBfield(['basecounter', [int, float], '(', 'counterfreq', False, False, None])),
                         ('siglen', WFDBfield(['siglen', [int], ' ', 'fs', False, True, None])),
                         ('basetime', WFDBfield(['basetime', [str], ' ', 'siglen', False, False, None])),
                         ('basedate', WFDBfield(['basedate', [str], ' ', 'basetime', False, False, None]))])
    
# 2. Signal specification fields
sigfields = OrderedDict([('filename', WFDBfield(['filename', [str], '', None, True, True, None])),
                         ('fmt', WFDBfield(['fmt', [int, str], ' ', 'filename', True, True, None])),
                         ('sampsperframe', WFDBfield(['sampsperframe', [int], 'x', 'fmt', False, False])),
                         ('skew', WFDBfield(['skew', [int], ':', 'fmt', False, False])),
                         ('byteoffset', WFDBfield(['byteoffset', [int], '+', 'fmt', False, False])),
                         ('adcgain', WFDBfield(['adcgain', [int], ' ', 'fmt', False, False])),
                         ('baseline', WFDBfield(['baseline', [int], '(', 'adcgain', False, False])),
                         ('units', WFDBfield(['units', [str], '/', 'adcgain', False, False])),
                         ('adcres', WFDBfield(['adcres', [int], ' ', 'adcgain', False, False])),
                         ('adczero', WFDBfield(['adczero', [int], ' ', 'adcres', False, False])),
                         ('initvalue', WFDBfield(['initvalue', [int], ' ', 'adczero', False, False])),
                         ('checksum', WFDBfield(['checksum', [int], ' ', 'initvalue', False, False])),
                         ('blocksize', WFDBfield(['blocksize', [int], ' ', 'checksum', False, False])),
                         ('signame', WFDBfield(['signame', [str], ' ', 'blocksize', False, False]))])
    
# 3. Segment specification fields
segfields = OrderedDict([('segname', WFDBfield(['segname', [str], '', None, True, True])),
                         ('seglen', WFDBfield(['seglen', [int], ' ', 'segname', True, True]))])
    
# 4. Comment fields
comfields = OrderedDict([('comments', WFDBfield(['comments', [int], '', None, False, False]))])

In [57]:
reqfields=OrderedDict()
for k, v in sigfields.items():
    if v.write_req:
        reqfields.update(OrderedDict([(k,v)]))


In [58]:
reqfields

OrderedDict([('filename', <__main__.WFDBfield at 0x7fcc743852b0>),
             ('fmt', <__main__.WFDBfield at 0x7fcc743852e8>)])

In [ ]:
d={1:4, 2:5}

In [88]:
x = [1, 2, 3, 'NOTNUM']
y = ['a', 'zzzz', 'HP', '99']

d={}

for i in range(0,len(x)):
    d[x[i]]=y[i]
    
print(d)
print(d['NOTNUM'])



{1: 'a', 2: 'zzzz', 3: 'HP', 'NOTNUM': '99'}
99


In [61]:
d1={'city': 'London', 'population': 15}


d1

{'city': 'London', 'population': 15}

In [102]:
keys = ['a', 'b', 9,  'c']
values = [1, 2, 3, 'PPP']
d = dict(zip(keys, values))

print(d)
print(d[9])

{'c': 'PPP', 'b': 2, 'a': 1, 9: 3}
3


In [1]:
qq.append(50)
qq

NameError: name 'qq' is not defined